So this function will be in a seperate class which are parametrized functions used to obtain the data from the GLEIF. That way each script can take care of and process the data invidually, but the same function logic for grabbing a link would not have to be repeated in each script.

In [4]:
from bs4 import BeautifulSoup
import os
import requests
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import sqlite3
import zipfile
import pandas as pd
import json
import pickle

class GLEIF_Data_Helpers:
    def __init__(self, bool_Level_1 = False, bool_Level_2_Trees = False, bool_Level_2_Reporting_Exceptions = False):
        self.bool_Level_1 = bool_Level_1
        self.bool_Level_2_Trees = bool_Level_2_Trees
        self.bool_Level_2_Reporting_Exceptions = bool_Level_2_Reporting_Exceptions

    def get_level_download_links(self):
        """
        This function uses selenium to webscrape the download link for all Level 1 Data in the GLEIF database.
        
        @return: str_download_link - the link which is used to download the entire GLEIF level 1
        """
        #Maybe new function

        driver_path = (r"C:\Drivers\Google\chromedriver-win64\chromedriver-win64\chromedriver.exe")
        service = Service(driver_path)
        driver = webdriver.Chrome(service=service)
        driver.get(url = "https://www.gleif.org/en/lei-data/gleif-golden-copy/download-the-golden-copy#/")

        cookie_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'CybotCookiebotDialogBodyButton'))
        )

        cookie_button.click()

        download_buttons = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'gc-download-button'))
        )
        
        if self.bool_Level_1 == True:
            download_buttons[0].click()
        if self.bool_Level_2_Trees == True:
            download_buttons[1].click()
        if self.bool_Level_2_Reporting_Exceptions == True:
            download_buttons[2].click()
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        driver.close()

        str_download_link = ((soup.find_all("a" , class_ = "gc-icon gc-icon--json"))[0])["href"]
        
        return str_download_link        
    
    def create_sql_instance(self, str_db_name, str_table_name):
        # Connect to the SQLite database with WAL mode enabled
        conn = sqlite3.connect(f'{str_db_name}.db', timeout=10)  # Set a timeout for waiting on locks
        conn.execute('PRAGMA journal_mode=WAL;')  # Enable WAL mode for concurrency
        cursor = conn.cursor()

        # Create the table with an id and JSON field (storing JSON as TEXT)
        cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {str_table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        data TEXT
        )
        ''')
        
        return conn, cursor
    
    def unpacking_GLEIF_zip_files(self , str_download_link , str_zip_file_path , str_unpacked_zip_file_path):
        session = requests.Session()
        zip_file = session.get(url = str_download_link)

        with open(str_zip_file_path, 'wb') as f:
            f.write(zip_file.content)

        with zipfile.ZipFile(str_zip_file_path, 'r') as zip_ref:
            os.makedirs(str_unpacked_zip_file_path, exist_ok=True)
            zip_ref.extractall(str_unpacked_zip_file_path)
        
        str_unpacked_zip_file_name = os.listdir(str_unpacked_zip_file_path)[0]
        str_json_file_path = str_unpacked_zip_file_path + "\\" + str_unpacked_zip_file_name
        
        return str_json_file_path

In [5]:
import bigjson
import json

class GLEIFLevel1Data:
    def __init__(self):
        self.str_level_1_unpacked_zip_file_path = r"C:\Users\mattp\Work_Related\Systematic_Trading\Library\Zip_Files\GLEIF\Level_1_Data\Unpacked_Zip"
        self.str_level_1_zip_file_path = r"C:\Users\mattp\Work_Related\Systematic_Trading\Library\Zip_Files\GLEIF\Level_1_Data.Level_1.zip"
        self.obj_data_helpers = GLEIF_Data_Helpers(bool_Level_1 = True)

    def insert_json_data(self, json_data , conn , cursor , str_table_name):
        cursor.execute(f'''
        INSERT INTO {str_table_name}  (data)
        VALUES (?)
        ''', (json.dumps(json_data),))
        conn.commit()
    
    def storing_GLEIF_data_in_database(self):
        str_level_1_download_link = self.obj_data_helpers.get_level_download_links()
        str_json_file_path = self.obj_data_helpers.unpacking_GLEIF_zip_files(str_download_link = str_level_1_download_link , str_zip_file_path = self.str_level_1_zip_file_path , str_unpacked_zip_file_path = self.str_level_1_unpacked_zip_file_path)
        conn, cursor = self.obj_data_helpers.create_sql_instance(str_table_name = "Level_1_Data" , str_db_name = "GLEIF_Data")
        
        with open(str_json_file_path, 'r' , encoding='utf-8') as file:
            test = bigjson.load(file)
            #counter = 1
            for dict_lei in test["records"]:
                #if counter != 15000:
                self.insert_json_data(json_data = dict_lei.to_python() , conn = conn , cursor = cursor , str_table_name = "Level_1_Data")
                    #counter += 1
                #else:
                    #break
                    
    def company_id_dictionary_generator(self):
        db_path = "GLEIF_Data.db"
        conn = sqlite3.connect(db_path)
        table_name = "Level_1_Data"  # Replace with your table name
        query = f"SELECT * FROM {table_name};"
        df = pd.read_sql_query(query, conn)
        conn.close()
        
        dict_company_names_leis = {}

        for _, row in df.iterrows():
            dict_company_data = json.loads(row.loc["data"])
            dict_company_names_leis[dict_company_data["Entity"]["LegalName"]["$"]] = dict_company_data["LEI"]["$"]
        
        return dict_company_names_leis

In [ ]:
obj_GLEIFLevel1Dara = GLEIFLevel1Data()
obj_GLEIFLevel1Dara.storing_GLEIF_data_in_database()

In [6]:
import pandas as pd

In [ ]:
def get_all_level_1_data():
    db_path = "GLEIF_Data.db"
    conn = sqlite3.connect(db_path)
    table_name = "Level_1_Data"  # Replace with your table name
    query = f"SELECT * FROM {table_name};"
    df_level_1_data = pd.read_sql_query(query, conn)
    conn.close()
    
    
    return df_level_1_data

#this will maybe be somewhere else
def company_name_id_dictionary_generator(df_level_1_data):
    dict_company_names_leis = {}

    for _, row in df_level_1_data.iterrows():
        dict_company_data = json.loads(row.loc["data"])
        dict_company_names_leis[dict_company_data["Entity"]["LegalName"]["$"]] = dict_company_data["LEI"]["$"]
    
    return dict_company_names_leis

df_level_1_data = get_all_level_1_data()
dict_company_names_leis = company_name_id_dictionary_generator(df_level_1_data = df_level_1_data)

In [8]:
with open(r"C:\Users\mattp\Work_Related\Systematic_Trading\Library\B_Notebooks\GLIEF_company_data_pipeline\pickled_objs\df_level_1_data.pickle" , "wb") as file:
    pickle.dump(df_level_1_data , file)

In [9]:
with open(r"C:\Users\mattp\Work_Related\Systematic_Trading\Library\B_Notebooks\GLIEF_company_data_pipeline\pickled_objs\dict_company_names_leis.pickle" , "wb") as file:
    pickle.dump(dict_company_names_leis , file)